In [84]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time

In [85]:
def parse_oneline(txt):
    output = [i for i in txt.split('\n')]
    return output

def parse_onepage(page_num):
    time.sleep(1) 
    html=requests.get(f"https://www.khug.or.kr/jeonse/web/s07/s070102.jsp?cur_page={page_num}")
    bs_test = BeautifulSoup(html.text, 'html.parser')
    tables = bs_test.find('table')
    thead = tables.find('thead')
    tab_cols = parse_oneline(thead.get_text().strip())
    tbody = tables.find('tbody')
    output_tab = pd.DataFrame([parse_oneline(i.strip()) for i in tbody.get_text().strip().split('\n\n')], columns=tab_cols)
    output_tab = output_tab.assign(href_id = np.unique([re.search('no=\d{10}', i['href']).group() for i in tbody.find_all(href=True)]))
    return output_tab

In [86]:
datas = [parse_onepage(n) for n in range(1,7)]

In [57]:
final_data = pd.concat(datas).reset_index(drop=True).assign(address = lambda df: df['주소'].apply(lambda x: x.split('  ')[0] if ',' not in x else  x.split('  ')[0].split(',')[0])
                                               # .apply(lambda x: ' '.join([i for i in x.split(' ') if i !='']))
                                              )

In [58]:
def find_coordinates(address):
    time.sleep(1)
    apiurl = "https://api.vworld.kr/req/address?"
    params = {
    	"service": "address",
    	"request": "getcoord",
    	"crs": "epsg:5179",
    	"address": address,
    	"format": "json",
    	"type": "parcel",
    	"key": "(api 키)"
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        result = response.json()['response']
        # print(result)
        if 'result' in result.keys():
            return result['result']['point']['x'], result['result']['point']['y']
        else:
            return 0,0

In [59]:
find_coordinates(final_data.address.values[0])

('947244.988988097', '1942135.319105934')

In [60]:
coordinates = [find_coordinates(i) for i in final_data.address.values]

In [61]:
final_data0 = pd.concat([final_data, pd.DataFrame(coordinates, columns = ['x','y'])], axis=1)

In [80]:
def get_img_link(href_id):
    time.sleep(1) 
    page_url = f"https://www.khug.or.kr/jeonse/web/s07/s070103.jsp?dt=20240830&{href_id}"
    html=requests.get(page_url)
    bs_test = BeautifulSoup(html.content, 'html.parser')
    img_src = bs_test.find(id = 'imgSor0')
    img_link = img_src.get('src')
    return img_link

In [82]:
final_data = final_data0.assign(img = lambda df: df.href_id.apply(lambda x: get_img_link(x)))

In [87]:
final_data.to_csv('data/test_data.csv', index = False, encoding = 'utf-8-sig')